In [ ]:
# install the Matatika client library
%%capture
!pip install matatika==1.5.0

In [ ]:
# configure a new client
from matatika.library import MatatikaClient

endpoint_url = 'https://catalog.matatika.com/api'
auth_token = '<YOUR AUTH TOKEN HERE>' # get from https://app.matatika.com/api-key
workspace_id = None

client = MatatikaClient(auth_token, endpoint_url, workspace_id)

In [ ]:
# fetch dataset data as Chart.js specification format
from matatika.types import DataFormat

dataset_id = '<YOUR DATASET ID HERE>'
data = client.fetch(dataset_id, data_format=DataFormat.CHARTJS)

In [ ]:
import pandas as pd

window_size = 7
threshold_percentage = 75

total_users_data = pd.Series(data['data']['datasets'][0]['data'])
rolling_average = total_users_data.rolling(window_size, center=True).mean()
upper = rolling_average * (1 + (threshold_percentage / 100))
lower = rolling_average * (1 - (threshold_percentage / 100))

data['data']['datasets'].append({
    'label': 'Rolling average',
    'borderWidth': 1,
    'fill': False,
    'backgroundColor': 'rgba(121, 209, 121, 0.2)',
    'borderColor': 'rgba(121, 209, 121, 1)',
    'borderWidth': 1,
    'data': rolling_average.where(upper.notnull(), None).to_list()
})

data['data']['datasets'].append({
    'label': 'Upper-threshold',
    'borderWidth': 1,
    'fill': '+1',
    'backgroundColor': 'rgba(255, 173, 20, 0.2)',
    'borderColor': 'rgba(255, 173, 20, 1)',
    'borderWidth': 1,
    'data': upper.where(upper.notnull(), None).to_list()
})

data['data']['datasets'].append({
    'label': 'Lower-threshold',
    'borderWidth': 1,
    'fill': -1,
    'backgroundColor': 'rgba(255, 173, 20, 0.2)',
    'borderColor': 'rgba(255, 173, 20, 1)',
    'borderWidth': 1,
    'data': lower.where(lower.notnull(), None).to_list()
})

In [ ]:
# install the Matatika IPlotter fork
%%capture
!pip install matatika-iplotter==0.4.4

# Total Website Users Over the Last 50 Days

Rolling average band plotted to visualise the days on which the total number of users were 75% higher or lower than a 7-day average.

- `Total Users` is the actual data
- `Rolling average` is a centered rolling average with a windows size of 7 elements
- `Upper-threshold` is the calculated rolling average with a 75% increase
- `Lower-threshold` is the calculated rolling average with a 75% decrease

In [ ]:
# generate a Chart.js chart from the fetched data
from iplotter import ChartJSPlotter

plotter = ChartJSPlotter()
plotter.plot(**data)